# Проект: Определение похожих товаров с использованием FAISS и оценка качества рекомендаций

## Описание проекта:

### Цель проекта

Создайть систему, способную идентифицировать и рекомендовать пять товаров, наиболее схожих с каждым элементом из проверочного датасета (validation.csv), используя для этого основную базу данных товаров (base.csv). Эффективность и точность предлагаемых рекомендаций необходимо оценить по метрике accuracy@5.

### Используемые технологии

* FAISS для быстрого поиска по схожести в больших наборах данных.
* Scikit-learn для обучения моделей и оценки качества рекомендаций.

### План работы

* Шаг 1: Подготовка данных и анализ
  * Загрузка и первичный анализ данных из base.csv, validation.csv и validation_answer.csv.
  * Преобразование идентификаторов Id и Target из строк в числовые значения для упрощения обработки.
  * Разделение признаков и целевых переменных, нормализация данных.
* Шаг 2: Работа с FAISS
  * Создание и конфигурация GPU-оптимизированного индекса в FAISS для базового набора данных.
  * Поиск топ-5 похожих товаров для каждого элемента в валидационном наборе.
* Шаг 3: Подготовка обучающего датасета
  * Формирование обучающего датасета на основе результатов поиска с использованием FAISS, включая создание признаков, отражающих схожесть между товарами.
  * Определение целевой переменной (Target) на основе правильных ответов из validation_answer.csv.
* Шаг 4: Обучение модей
  * Оценка качества модели с использованием кросс-валидации и метрики accuracy@5.
* Шаг 5: Анализ результатов
  * Детальный анализ результатов предсказания модели.
  * Сравнение эффективности рекомендаций FAISS и точности классификации модели логистической регрессии.

## Подготовка данных

### Загрузка библиотек, таблиц, инициализация функций и констант

In [7]:
# Базовые библиотеки для работы с данными
import pandas as pd
import numpy as np

# Визуализация
import seaborn as sns
import matplotlib.pyplot as plt

# Модели и инструменты машинного обучения
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
import lightgbm as lgb


# Утилиты
from IPython.display import display
import warnings
import time

# Библиотеки для анализа данных
import phik
from phik import report

# Игнорирование предупреждений
warnings.filterwarnings('ignore')

In [8]:
pd.set_option('display.min_rows', 15)
pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.2f}'.format

In [9]:
valid = pd.read_csv('validation.csv')

valid_target= pd.read_csv('validation_answer.csv')

train = pd.read_csv('train.csv')

base = pd.read_csv('base.csv')

In [10]:
# функция информации по таблице
def dataframe_summary(df, string):
    # Вывод общей информации
    print("Общая информация по таблице:", string)
    df.info()

    print("\n Статистическое описание:")
    display(df.describe().transpose())

    print("\nСлучайные примеры:")
    display(df.sample(5))

    print("\nКоличество строк и столбцов:", df.shape)

    print("\nКоличество явных дубликатов:", df.duplicated().sum())
    print('')

In [11]:
def downcast_dataframe(df):

    # Цикл по всем столбцам DataFrame
    for col in df.columns:
        if df[col].dtype == 'float64':  # Проверяем, является ли тип столбца float64
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif df[col].dtype == 'int64':  # Проверяем, является ли тип столбца int64
            df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

Все данные загружены, можно приступать к работе.

### Обзор данных

In [12]:
#for data in [valid, valid_target, train, base]:
#    dataframe_summary(data, 'Данные')

* В данных отсутсвуют пропуски и дубликаты. 
* Наблюдается сильный разброс значений, понадобится масштабирование.
* Стоит отменить, что в `valid_target` в колонке `Expected` 91502 уникальных значений из 100000 строк. Значит есть товары, которые указали как схожие несколько раз.

Уменьшим потребляемый объем памяти данных. Переведем столбец Id в формат индекса. И уменьшим регистр называний столбцов.

In [13]:
base = downcast_dataframe(base)
base.columns = base.columns.str.lower()
base.set_index('id', inplace=True)

train = downcast_dataframe(train)
train.columns = train.columns.str.lower()
train.set_index('id', inplace=True)

valid = downcast_dataframe(valid)
valid.columns = valid.columns.str.lower()
valid.set_index('id', inplace=True)


In [14]:
valid_target = downcast_dataframe(valid_target)
valid_target.columns = valid_target.columns.str.lower()
valid_target.set_index('id', inplace=True)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0-query to 99999-query
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float32
 1   1       100000 non-null  float32
 2   2       100000 non-null  float32
 3   3       100000 non-null  float32
 4   4       100000 non-null  float32
 5   5       100000 non-null  float32
 6   6       100000 non-null  float32
 7   7       100000 non-null  float32
 8   8       100000 non-null  float32
 9   9       100000 non-null  float32
 10  10      100000 non-null  float32
 11  11      100000 non-null  float32
 12  12      100000 non-null  float32
 13  13      100000 non-null  float32
 14  14      100000 non-null  float32
 15  15      100000 non-null  float32
 16  16      100000 non-null  float32
 17  17      100000 non-null  float32
 18  18      100000 non-null  float32
 19  19      100000 non-null  float32
 20  20      100000 non-null  float32
 21  21  

Потребляемый объем памяти упал почти в два раза.

In [16]:
# выделение цел. признака из обучающей выборки
train_target = train['target']
train.drop('target', axis=1, inplace=True)

In [17]:
# Отбор 30% данных из base случайным образом
sampled_base_df = base.sample(frac=0.3, random_state=42) 

In [18]:
# словарь id и номеров векторов
base_index = {k: v for k, v in enumerate(base.index.to_list())}
train_index = {k: v for k, v in enumerate(train.index.to_list())}

In [19]:
base_cols = base.columns

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
import torch.cuda as cuda
import torch

In [22]:
cuda.is_available()

True

In [23]:
print(torch.__version__)

2.2.2


In [ ]:
print(torch.version.cuda)  # Выводит версию CUDA, с которой скомпилирован PyTorch
print(torch.cuda.get_device_name(0))  # Выводит имя вашего GPU

In [24]:
# Инициализация масштабировщика
scaler = StandardScaler()

# Обучение масштабировщика на данных из base
scaler.fit(base)

# Масштабирование данных base, train и valid
base = scaler.transform(base)
train = scaler.transform(train)
valid = scaler.transform(valid)

In [25]:
base

array([[-1.1592164 ,  0.6203504 , -0.5137226 , ..., -0.0140508 ,
         1.7814199 , -0.31232867],
       [ 2.0757148 ,  1.0604233 , -0.65249103, ...,  0.05984759,
         1.8537259 , -0.28105187],
       [ 1.2854173 , -0.3433421 ,  0.39787757, ...,  0.04852088,
        -0.71384674,  0.36562327],
       ...,
       [-0.4337762 , -2.064035  , -0.69096935, ...,  0.6597316 ,
        -0.71384674,  1.2577736 ],
       [-0.02446471,  0.16793932,  0.25220424, ...,  0.43807355,
        -0.71384674, -0.19157949],
       [-0.6321803 ,  0.96487993, -0.17634065, ..., -1.0735649 ,
        -0.7121896 ,  1.4986584 ]], dtype=float32)

In [26]:
import faiss

In [ ]:
# Определяем количество кластеров
N_CLUSTERS = 200  # Пример количества кластеров

d = base.shape[1]  # Размерность векторов признаков

# Создание квантизатора для разделения векторного пространства на кластеры
quantizer = faiss.IndexFlatL2(d)

# Создание индекса IndexIVFFlat с использованием квантизатора
idx_l2 = faiss.IndexIVFFlat(quantizer, d, N_CLUSTERS, faiss.METRIC_L2)

# Инициализация ресурсов GPU
res = faiss.StandardGpuResources()

# Обучение индекса на базовом наборе данных
if not idx_l2.is_trained:
    idx_l2.train(base)

# Создание GPU-версии индекса из CPU-версии
gpu_index = faiss.index_cpu_to_gpu(res, 0, idx_l2)

# Добавление данных в индекс
gpu_index.add(base)

# Поиск ближайших соседей для данных из valid
k = 40  # Количество ближайших соседей для поиска
D, I = gpu_index.search(valid, k)  # D - расстояния, I - индексы ближайших соседей

# Вывод индексов ближайших соседей и соответствующих расстояний
print("Индексы ближайших соседей:\n", I)
print("Расстояния до ближайших соседей:\n", D)

In [ ]:
# Создание индекса с использованием GPU
d = base.shape[1]  # Размерность векторов
res = faiss.StandardGpuResources()  # инициализация ресурсов GPU
index = faiss.IndexFlatL2(d)  # создание L2 индекса
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)  # перенос индекса на GPU

# Добавление данных в индекс
gpu_index.add(base)

# Поиск 5 ближайших соседей для первых 10 векторов
k = 400
D, I = gpu_index.search(base[:10], k)

print("Индексы ближайших соседей для первых 10 векторов:\n", I)
print("Расстояния до ближайших соседей для первых 10 векторов:\n", D)

In [ ]:
# Определение размерности векторов
d = base.shape[1]

# Создание индекса для использования на GPU
res = faiss.StandardGpuResources()  # Инициализация ресурсов GPU
index = faiss.IndexFlatL2(d)  # Создание индекса L2
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)  # Перемещение индекса на GPU

# Добавление данных из base в индекс
gpu_index.add(base)

# Поиск 5 ближайших соседей для каждого вектора из valid
k = 5  # Количество ближайших соседей для поиска
D, I = gpu_index.search(valid, k)  # D - расстояния, I - индексы ближайших соседей

# Теперь в I содержатся индексы ближайших соседей из base для каждого элемента из valid
print("Индексы ближайших соседей:\n", I)
print("Расстояния до ближайших соседей:\n", D)